In [9]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report
import numpy as np
import os

In [10]:
def load_mnist():
    (train_X, train_y), (test_X, test_y) = mnist.load_data()
    train_X = train_X.reshape(-1, 784)
    test_X = test_X.reshape(-1, 784)
    train_X = train_X.astype('float32')
    test_X = test_X.astype('float32')
    train_X /= 255
    test_X /= 255

    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    return {"train_X": train_X[:55000, :], "train_y": train_y[:55000, :],
            "val_X": train_X[55000:, :], "val_y": train_y[55000:, :], "test_X": test_X, "test_y": test_y}

In [11]:
def build_network(input_features=None):
    # first we specify an input layer, with a shape == features
    inputs = Input(shape=(input_features,), name="input")
    x = Dense(512, activation='relu', name="hidden1")(inputs)
    x = Dense(256, activation='relu', name="hidden2")(x)
    x = Dense(128, activation='relu', name="hidden3")(x)
    prediction = Dense(10, activation='softmax', name="output")(x)
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [12]:
def create_callbacks():
    tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "tb_log", "mnist_512_256_128"), histogram_freq=1, batch_size=32,
                                       write_graph=True, write_grads=False)
    checkpoint_callback = ModelCheckpoint(filepath="./model-weights.{epoch:02d}-{val_accuracy:.6f}.hdf5", monitor='val_acc',
                                          verbose=0, save_best_only=True)
    return [tensorboard_callback, checkpoint_callback]

In [13]:
def print_model_metrics(model, data):
    loss, accuracy = model.evaluate(x=data["test_X"], y=data["test_y"])
    print("\n model test loss is "+str(loss)+" accuracy is "+str(accuracy))

    y_softmax = model.predict(data["test_X"])  # this is an n x class matrix of probabilities
    y_hat = y_softmax.argmax(axis=-1)  # this will be the class number.
    test_y = data["test_y"].argmax(axis=-1)  # our test data is also categorical
    print(classification_report(test_y, y_hat))

In [14]:
def main():
    data = load_mnist()
    callbacks = create_callbacks()
    model = build_network(data["train_X"].shape[1])
    model.fit(x=data["train_X"], y=data["train_y"],
              batch_size=30,
              epochs=50,
              validation_data=(data["val_X"], data["val_y"]),
              verbose=1,
              callbacks=callbacks)

    print_model_metrics(model, data)

In [15]:
main()

Epoch 1/50
1834/1834 [==============================] - 8s 4ms/step - loss: 0.2011 - accuracy: 0.9383 - val_loss: 0.1361 - val_accuracy: 0.9610
Epoch 2/50
1834/1834 [==============================] - 6s 3ms/step - loss: 0.0916 - accuracy: 0.9721 - val_loss: 0.0835 - val_accuracy: 0.9762
Epoch 3/50
1834/1834 [==============================] - 6s 3ms/step - loss: 0.0647 - accuracy: 0.9802 - val_loss: 0.0770 - val_accuracy: 0.9788
Epoch 4/50
1834/1834 [==============================] - 6s 3ms/step - loss: 0.0508 - accuracy: 0.9839 - val_loss: 0.0870 - val_accuracy: 0.9756
Epoch 5/50
1834/1834 [==============================] - 10s 5ms/step - loss: 0.0392 - accuracy: 0.9869 - val_loss: 0.0905 - val_accuracy: 0.9764
Epoch 6/50
1834/1834 [==============================] - 11s 6ms/step - loss: 0.0347 - accuracy: 0.9884 - val_loss: 0.0818 - val_accuracy: 0.9800
Epoch 7/50
1834/1834 [==============================] - 11s 6ms/step - loss: 0.0281 - accuracy: 0.9916 - val_loss: 0.0779 - val_accura